In [ ]:
import requests
import pandas as pd
import time
import folium

In [44]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Read the geojson file
geo_taxi_zone = gpd.read_file('../../datasets/raw/taxi_raw/NYC_Taxi_Zones.geojson')
geo_taxi_zone

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [46]:
#calculate centroids for each taxi zone
geo_taxi_zone['centroid'] = geo_taxi_zone.geometry.centroid

#latitude and longitude from the centroids
geo_taxi_zone['latitude'] = geo_taxi_zone.centroid.y
geo_taxi_zone['longitude'] = geo_taxi_zone.centroid.x
geo_taxi_zone.head()

C:\Users\natha\AppData\Local\Temp\ipykernel_39412\3415677558.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['centroid'] = geo_taxi_zone.geometry.centroid
C:\Users\natha\AppData\Local\Temp\ipykernel_39412\3415677558.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['latitude'] = geo_taxi_zone.centroid.y
C:\Users\natha\AppData\Local\Temp\ipykernel_39412\3415677558.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_taxi_zone['longitude'] = geo_taxi_zone.centroid.x


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,centroid,latitude,longitude
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ...",POINT (-74.17400 40.69183),40.691831,-74.174000
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",POINT (-73.83130 40.61675),40.616745,-73.831299
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",POINT (-73.84742 40.86447),40.864474,-73.847422
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",POINT (-73.97697 40.72375),40.723752,-73.976968
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ...",POINT (-74.18848 40.55266),40.552659,-74.188484


In [54]:
# Set your Google Places API key here
api_key = 'XXX'

def get_attractions(lat, lng, radius=500):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=tourist_attraction&key={api_key}"
    response = requests.get(url)
    return response.json()

#list to store attraction data
attraction_data = []

#loop through each taxi zone to get attractions
for index, row in geo_taxi_zone.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    response = get_attractions(lat, lng)
    attractions = response.get('results', [])[:5]  # Get top 5 attractions

    for attraction in attractions:
        attraction_data.append({
            'taxi_zone': row['location_id'],
            'zone_name': row['zone'],
            'attraction_name': attraction['name'],
            'attraction_latitude': attraction['geometry']['location']['lat'],
            'attraction_longitude': attraction['geometry']['location']['lng'],
            'attraction_vicinity': attraction.get('vicinity'),
            'attraction_rating': attraction.get('rating'),
            'user_ratings_total': attraction.get('user_ratings_total'),
        })
    
    #avoid hitting API rate limits
    time.sleep(1)


attractions_df = pd.DataFrame(attraction_data)
attractions_df.head()

,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total
0,4,Alphabet City,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,185.0
1,4,Alphabet City,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0
2,4,Alphabet City,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0
3,4,Alphabet City,Orchard Alley,40.721375,-73.978441,"350 East 4th Street, New York",4.2,17.0
4,4,Alphabet City,Museum of Reclaimed Urban Space (MoRUS),40.725766,-73.977946,"155 Loisaida Avenue, New York",4.8,89.0


In [55]:
attractions_df

,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total
0,4,Alphabet City,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,185.0
1,4,Alphabet City,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0
2,4,Alphabet City,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0
3,4,Alphabet City,Orchard Alley,40.721375,-73.978441,"350 East 4th Street, New York",4.2,17.0
4,4,Alphabet City,Museum of Reclaimed Urban Space (MoRUS),40.725766,-73.977946,"155 Loisaida Avenue, New York",4.8,89.0
...,...,...,...,...,...,...,...,...
569,261,World Trade Center,Federal Reserve Bank of New York,40.708364,-74.008653,"33 Liberty Street, New York",4.1,716.0
570,262,Yorkville East,Gracie Mansion Conservancy,40.776144,-73.943113,"E 88th St &, East End Avenue, New York",4.3,102.0
571,262,Yorkville East,Carl Schurz Park,40.775130,-73.943697,"East 86th Street &, East End Avenue, New York",4.7,3164.0
572,262,Yorkville East,Catbird Playground,40.774015,-73.944686,"Carl Schurz Park, East End Avenue, New York",4.7,115.0


In [56]:
attractions_df.isnull().sum()

taxi_zone                0
zone_name                0
attraction_name          0
attraction_latitude      0
attraction_longitude     0
attraction_vicinity      0
attraction_rating       29
user_ratings_total      29
dtype: int64

In [57]:
# Clean duplicated rows
print('Number of duplicate (excluding original) rows is:', attractions_df.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', attractions_df[attractions_df.duplicated(keep=False)].shape[0])
# Show duplicate row data that can be dropped
attractions_df[attractions_df.duplicated(keep=False)]

Number of duplicate (excluding original) rows is: 0
Number of duplicate rows (including first) in the table is: 0


,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total


In [58]:
# show date types
attractions_df.dtypes

taxi_zone                object
zone_name                object
attraction_name          object
attraction_latitude     float64
attraction_longitude    float64
attraction_vicinity      object
attraction_rating       float64
user_ratings_total      float64
dtype: object

In [59]:
attractions_df = attractions_df.merge(geo_taxi_zone[['objectid', 'borough']], 
                               left_on='taxi_zone', 
                               right_on='objectid', 
                               how='left')

# drop the 'objectid' column not needed
attractions_df.drop('objectid', axis=1, inplace=True)

In [75]:
attractions_df

,taxi_zone,zone_name,attraction_name,attraction_latitude,attraction_longitude,attraction_vicinity,attraction_rating,user_ratings_total,borough
0,4,Alphabet City,6BC Botanical Garden,40.723815,-73.980708,"630 East 6th Street, New York",4.6,185.0,Manhattan
1,4,Alphabet City,John V. Lindsay East River Park Track,40.721697,-73.973436,"1234 East 6th Street, New York",4.6,880.0,Manhattan
2,4,Alphabet City,Green Oasis,40.724047,-73.977378,"370 East 8th Street, New York",4.7,37.0,Manhattan
3,4,Alphabet City,Orchard Alley,40.721375,-73.978441,"350 East 4th Street, New York",4.2,17.0,Manhattan
4,4,Alphabet City,Museum of Reclaimed Urban Space (MoRUS),40.725766,-73.977946,"155 Loisaida Avenue, New York",4.8,89.0,Manhattan
...,...,...,...,...,...,...,...,...,...
569,261,World Trade Center,Federal Reserve Bank of New York,40.708364,-74.008653,"33 Liberty Street, New York",4.1,716.0,Manhattan
570,262,Yorkville East,Gracie Mansion Conservancy,40.776144,-73.943113,"E 88th St &, East End Avenue, New York",4.3,102.0,Manhattan
571,262,Yorkville East,Carl Schurz Park,40.775130,-73.943697,"East 86th Street &, East End Avenue, New York",4.7,3164.0,Manhattan
572,262,Yorkville East,Catbird Playground,40.774015,-73.944686,"Carl Schurz Park, East End Avenue, New York",4.7,115.0,Manhattan


In [74]:
attractions_df.to_csv("attractions_df.csv", index=False)

In [61]:
#unique values for each column
unique_counts = attractions_df.nunique()

#print the unique counts for each column
print("Unique counts for each column:\n", unique_counts)

Unique counts for each column:
 taxi_zone               189
zone_name               189
attraction_name         525
attraction_latitude     526
attraction_longitude    524
attraction_vicinity     498
attraction_rating        21
user_ratings_total      355
borough                   5
dtype: int64


In [65]:
attractions_df = attractions_df[attractions_df['borough'] == 'Manhattan']
attractions_df.shape

(306, 9)

In [73]:
# map centered around Manhattan
m = folium.Map(location=[40.7831, -73.9712], zoom_start=12)

geo_taxi_zone = gpd.read_file('../../datasets/raw/taxi_raw/NYC_Taxi_Zones.geojson')
geo_taxi_zone = geo_taxi_zone[geo_taxi_zone['borough'] == 'Manhattan']

#add taxi zones polygons to the map
folium.GeoJson(geo_taxi_zone, name="Taxi Zones").add_to(m)

#iterate over each taxi zone
for index, row in geo_taxi_zone.iterrows():
    # Get the centroid of the taxi zone
    centroid = row['geometry'].centroid
    lat = centroid.y
    lng = centroid.x

#add POIs to the map
for index, row in attractions_df.iterrows():
    folium.Marker([row['attraction_latitude'], row['attraction_longitude']],
                  popup=row['attraction_name'],
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(m)

#save the map to an HTML file
m.save('manhattan_poi_map.html')
print("Map with taxi zones and POIs has been saved as 'manhattan_poi_map.html'")

Map with taxi zones and POIs has been saved as 'manhattan_poi_map.html'
